In [ ]:
from stravalib.client import Client
from stravalib.model import Activity
import csv
import datetime
import os
import sys
import inspect
import datetime
currentdir = os.path.dirname(os.path.abspath(
    inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
# Include paths for module search
sys.path.insert(0, parentdir)
from db import ChallengeDB
from model import Run, Runner

# Required ENV
CLIENT_ID = os.environ["STRAVA_CLIENT_ID"]
CLIENT_SECRET = os.environ["STRAVA_CLIENT_SECRET"]

MONGODB_URI = os.environ["MONGODB_URI"]
DATABASE_NAME = os.environ["DATABASE_NAME"]

In [ ]:
now = datetime.datetime.now()
timestamp = now.strftime("%Y-%m-%d %H:%M:%S")
report_postfix = now.strftime("%Y%m%d_%H%M%S")
print(timestamp)

In [ ]:
ChallengeDB.init(MONGODB_URI, DATABASE_NAME)

In [ ]:
runs = ChallengeDB.find_run()

In [14]:
runners = ChallengeDB.find_runner()

In [ ]:
for run in runs:
    print(run.to_doc())

In [ ]:
for runner in runners:
    print(runner.to_doc())

In [15]:
run_report_path = "run_report_%s.csv" % (report_postfix)
with open(run_report_path, 'w', newline='') as csvfile:
    fieldnames = ["timestamp"] + list(runs[0].to_doc().keys())
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for run in runs:
        row = run.to_doc()
        row["timestamp"] = timestamp
        writer.writerow(row)